In [1]:
from scripts.data_util.sqlConnector import online_sql_connector , update_sql_since
connector = online_sql_connector()
update_sql_since('SellSideFactors' , connector = connector , trace = 0)
connector.close_all()

Sun Feb  4 08:20:46 2024 : haitong/hf_factors since 20240127 to 20240204, total 1 periods(Q)
Sun Feb  4 08:20:55 2024 : haitong/dl_factors since 20240127 to 20240204, total 1 periods(Q)
Sun Feb  4 08:21:43 2024 : dongfang/hfq_chars since 20240127 to 20240204, total 1 periods(Q)
Sun Feb  4 08:21:51 2024 : dongfang/l2_chars since 20240127 to 20240204, total 1 periods(Q)
Sun Feb  4 08:22:22 2024 : dongfang/ms_chars since 20240127 to 20240204, total 1 periods(Q)
Sun Feb  4 08:24:23 2024 : dongfang/order_flow since 20240127 to 20240204, total 1 periods(Q)
Sun Feb  4 08:24:26 2024 : dongfang/gp since 20240127 to 20240204, total 1 periods(Q)
Sun Feb  4 08:24:30 2024 : dongfang/tra since 20240127 to 20240204, total 1 periods(Q)
Sun Feb  4 08:24:33 2024 : dongfang/scores_v0 since 20240127 to 20240204, total 1 periods(Q)
Sun Feb  4 08:24:34 2024 : kaiyuan/positive since 20240101 to 20240204, total 1 periods(Q)
Sun Feb  4 08:24:39 2024 : kaiyuan/negative since 20231230 to 20240204, total 1 period

In [3]:
#from scripts.data_util.DataTank import *
#from scripts.data_util.DataUpdater import *
#repack_DB_bygroup('data/DB_data/DB_3rdPartySQL.h5' , 'SellSideFactors' , True)